# TD Count-Length Experimenter

This sandbox lets you sweep the **Setup bar count** and **Countdown bar count** thresholds parametrically, rather than being locked to the traditional 9-bar setup and 13-bar countdown.

The key question: *Are 9 and 13 actually optimal for these instruments, or is that Demark folklore?*

## What can be varied
| Parameter | Original | What it controls |
|---|---|---|
| `SETUP_COUNT` | 9 | Number of consecutive qualifying bars to trigger a setup |
| `COUNTDOWN_COUNT` | 13 | Number of qualifying countdown bars to trigger entry |
| `SETUP_LOOKBACK` | 4 | `close < close[N bars ago]` — the comparison offset |
| `COUNTDOWN_LOOKBACK` | 2 | `close <= low[N bars ago]` — the countdown comparison offset |
| `CARRY_WINDOW` | 63 (setup) / 126 (cd) | Max bars to hold a position |

## Cells
1. Installs & Imports
2. Config & Data
3. Parameterised TD Engine
4. Single-Config Backtest
5. Grid Search (sweep setup & countdown counts)
6. Results Dashboard
7. Best-Config Deep Dive
8. Comparison vs Original 9/13

# Cell 0 — Installs & Imports

In [ ]:
# Cell 0: Installs & Imports
# ============================================================
!pip install yfinance --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from itertools import product
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', '{:.4f}'.format)
plt.style.use('seaborn-v0_8-darkgrid')
print("Imports complete")

# Cell 1 — Config & Data

In [ ]:
# Cell 1: Config & Data
# ============================================================

TICKERS    = ['SPY', 'QQQ', 'IWM', 'TLT', 'HYG',
              'GLD', 'USO', 'UUP', 'EEM', 'VNQ', 'SLV', 'XLF', 'XLE']
START_DATE = '2015-01-01'
COST_BPS   = 5

# ── DEFAULT SINGLE-RUN PARAMETERS ─────────────────────────
# Edit these to test a specific configuration before running
# the full grid search.
DEFAULT_SETUP_COUNT       = 9    # Original: 9
DEFAULT_COUNTDOWN_COUNT   = 13   # Original: 13
DEFAULT_SETUP_LOOKBACK    = 4    # Original: 4   (close < close[-N])
DEFAULT_COUNTDOWN_LOOKBACK= 2    # Original: 2   (close <= low[-N])
DEFAULT_CARRY_SETUP       = 63   # Original: 63  bars to hold setup signal
DEFAULT_CARRY_COUNTDOWN   = 126  # Original: 126 bars to hold countdown signal

# ── GRID SEARCH RANGES ────────────────────────────────────
# The full grid search will test every combination of these.
# Tip: keep ranges small at first — a 5x5 grid over 13 tickers
# is already 325 backtests.
SETUP_COUNT_RANGE      = [7, 8, 9, 10, 11, 12]    # bars for setup completion
COUNTDOWN_COUNT_RANGE  = [9, 10, 11, 12, 13, 14, 15]  # bars for countdown
# The lookback and carry params are kept fixed during the grid
# search but you can uncomment ranges to sweep those too.
# SETUP_LOOKBACK_RANGE   = [3, 4, 5]
# COUNTDOWN_LOOKBACK_RANGE = [1, 2, 3]

# Fetch data
print("Fetching OHLCV...")
raw = yf.download(
    TICKERS,
    start      = START_DATE,
    auto_adjust= True,
    progress   = False
)

close   = raw['Close'].dropna(how='all').ffill(limit=3)
high    = raw['High'].dropna(how='all').ffill(limit=3)
low     = raw['Low'].dropna(how='all').ffill(limit=3)
returns = close.pct_change()

print(f"  Shape : {close.shape}")
print(f"  Range : {close.index[0].date()} -> {close.index[-1].date()}")
print("Data ready")

# Cell 2 — Helper Functions

In [ ]:
# Cell 2: Helper Functions
# ============================================================

def apply_costs(returns, position, cost_bps=5):
    cost     = cost_bps / 10_000
    turnover = position.diff().abs()
    return returns * position - turnover * cost


def performance_summary(r, name=''):
    r   = r.dropna()
    if len(r) == 0:
        return pd.Series({'Ann. Return': np.nan, 'Ann. Vol': np.nan,
                          'Sharpe': np.nan, 'Max DD': np.nan,
                          'Calmar': np.nan, 'Hit Rate': np.nan}, name=name)
    cum = (1 + r).cumprod()
    ann = 252
    ann_ret = (1 + r.mean()) ** ann - 1
    ann_vol = r.std() * np.sqrt(ann)
    sharpe  = ann_ret / ann_vol if ann_vol > 0 else np.nan
    roll_max = cum.cummax()
    mdd     = (cum / roll_max - 1).min()
    calmar  = ann_ret / abs(mdd) if mdd != 0 else np.nan
    hits    = (r > 0).mean()
    return pd.Series({
        'Ann. Return': ann_ret,
        'Ann. Vol':    ann_vol,
        'Sharpe':      sharpe,
        'Max DD':      mdd,
        'Calmar':      calmar,
        'Hit Rate':    hits,
    }, name=name)


def signal_density(sig_df):
    """Fraction of ticker-days where the signal is active."""
    return sig_df.mean().mean()


print("Helpers loaded")

# Cell 3 — Parameterised TD Engine

In [ ]:
# Cell 3: Parameterised TD Engine
# ============================================================
# All count lengths are passed as arguments — nothing hardcoded.
#
# Key parameters explained:
#
#   setup_count       : How many consecutive qualifying bars needed
#                       to complete a setup (original = 9)
#
#   countdown_count   : How many qualifying countdown bars fire the
#                       entry signal (original = 13)
#
#   setup_lookback    : The 'N' in close < close[-N] for the setup
#                       condition (original = 4)
#
#   countdown_lookback: The 'N' in close <= low[-N] for the countdown
#                       condition (original = 2)
#
#   carry_setup       : Rolling window used to carry a setup signal
#                       forward until TDST exit (bars, original = 63)
#
#   carry_countdown   : Same for countdown signal (bars, original = 126)


def rolling_consecutive_count(cond: pd.DataFrame) -> pd.DataFrame:
    """
    Convert a binary condition DataFrame into a consecutive-count
    DataFrame that resets to zero on False.
    """
    counts = pd.DataFrame(0, index=cond.index, columns=cond.columns)
    for col in cond.columns:
        s        = cond[col]
        group_id = (s == 0).cumsum()
        raw_count = s.groupby(group_id).cumsum()
        counts[col] = raw_count * s
    return counts


def compute_td_setup_param(
        close: pd.DataFrame,
        high: pd.DataFrame,
        low: pd.DataFrame,
        setup_count: int = 9,
        setup_lookback: int = 4) -> dict:
    """
    Parameterised TD Setup.

    Bullish: close < close[setup_lookback bars ago]  for setup_count consecutive bars
    Bearish: close > close[setup_lookback bars ago]  for setup_count consecutive bars

    Returns:
        bull_count       : running count per bar
        bear_count       : running count per bar
        bull_setup_signal: fires on bar == setup_count
        bear_setup_signal: fires on bar == setup_count
    """
    close_n = close.shift(setup_lookback)

    bull_cond = (close < close_n).astype(int)
    bear_cond = (close > close_n).astype(int)

    bull_count = rolling_consecutive_count(bull_cond)
    bear_count = rolling_consecutive_count(bear_cond)

    bull_signal = (bull_count == setup_count).astype(int)
    bear_signal = (bear_count == setup_count).astype(int)

    # TDST levels
    lowest_n  = low.rolling(setup_count).min()
    highest_n = high.rolling(setup_count).max()

    tdst_support = pd.DataFrame(np.nan, index=close.index, columns=close.columns)
    tdst_resistance = pd.DataFrame(np.nan, index=close.index, columns=close.columns)

    tdst_support[bull_signal == 1]    = lowest_n[bull_signal == 1]
    tdst_resistance[bear_signal == 1] = highest_n[bear_signal == 1]

    tdst_support    = tdst_support.ffill()
    tdst_resistance = tdst_resistance.ffill()

    tdst_long_valid  = (close > tdst_support).astype(int)
    tdst_short_valid = (close < tdst_resistance).astype(int)

    return {
        'bull_count':        bull_count,
        'bear_count':        bear_count,
        'bull_setup_signal': bull_signal,
        'bear_setup_signal': bear_signal,
        'tdst_support':      tdst_support,
        'tdst_resistance':   tdst_resistance,
        'tdst_long_valid':   tdst_long_valid,
        'tdst_short_valid':  tdst_short_valid,
    }


def compute_td_countdown_param(
        close: pd.DataFrame,
        high: pd.DataFrame,
        low: pd.DataFrame,
        bull_setup_signal: pd.DataFrame,
        bear_setup_signal: pd.DataFrame,
        countdown_count: int = 13,
        countdown_lookback: int = 2) -> dict:
    """
    Parameterised TD Countdown with recycling.

    Bullish condition: close <= low[countdown_lookback bars ago]
    Bearish condition: close >= high[countdown_lookback bars ago]

    countdown_count controls when the signal fires.
    Recycling: same-direction setup resets the countdown.
    Cancellation: opposite setup cancels the countdown.
    """
    low_n  = low.shift(countdown_lookback)
    high_n = high.shift(countdown_lookback)

    bull_cd_cond = (close <= low_n)
    bear_cd_cond = (close >= high_n)

    bull_cd_count  = pd.DataFrame(0, index=close.index, columns=close.columns)
    bear_cd_count  = pd.DataFrame(0, index=close.index, columns=close.columns)
    bull_cd_active = pd.DataFrame(0, index=close.index, columns=close.columns)
    bear_cd_active = pd.DataFrame(0, index=close.index, columns=close.columns)

    n_bars = len(close)

    for col in close.columns:
        # Bullish countdown
        in_cd = False
        b_count = 0
        b_counts = [0] * n_bars
        b_active = [0] * n_bars

        for i in range(n_bars):
            if bull_setup_signal[col].iloc[i] == 1:
                b_count = 0
                in_cd   = True
            if bear_setup_signal[col].iloc[i] == 1:
                in_cd   = False
                b_count = 0
            if in_cd and b_count < countdown_count:
                if bull_cd_cond[col].iloc[i]:
                    b_count += 1
            b_counts[i] = b_count if in_cd else 0
            b_active[i] = 1     if in_cd else 0

        bull_cd_count[col]  = b_counts
        bull_cd_active[col] = b_active

        # Bearish countdown
        in_cd   = False
        r_count = 0
        r_counts = [0] * n_bars
        r_active = [0] * n_bars

        for i in range(n_bars):
            if bear_setup_signal[col].iloc[i] == 1:
                r_count = 0
                in_cd   = True
            if bull_setup_signal[col].iloc[i] == 1:
                in_cd   = False
                r_count = 0
            if in_cd and r_count < countdown_count:
                if bear_cd_cond[col].iloc[i]:
                    r_count += 1
            r_counts[i] = r_count if in_cd else 0
            r_active[i] = 1     if in_cd else 0

        bear_cd_count[col]  = r_counts
        bear_cd_active[col] = r_active

    bull_cd_signal = (bull_cd_count == countdown_count).astype(int)
    bear_cd_signal = (bear_cd_count == countdown_count).astype(int)

    return {
        'bull_cd_count':  bull_cd_count,
        'bear_cd_count':  bear_cd_count,
        'bull_cd_signal': bull_cd_signal,
        'bear_cd_signal': bear_cd_signal,
        'bull_cd_active': bull_cd_active,
        'bear_cd_active': bear_cd_active,
    }


print("Parameterised TD engine loaded")

# Cell 4 — Signal Builders & Backtest Function

In [ ]:
# Cell 4: Signal Builders & Single-Config Backtest
# ============================================================

def build_signals_for_config(
        close, high, low,
        setup_count       = 9,
        countdown_count   = 13,
        setup_lookback    = 4,
        countdown_lookback= 2,
        carry_setup       = 63,
        carry_countdown   = 126):
    """
    Build all TD signals for a given parameter config.
    Returns a dict with long/short signals for:
      - setup only (V1)
      - countdown only (V2)
      - both combined (V3)
    """
    td = compute_td_setup_param(
        close, high, low,
        setup_count=setup_count,
        setup_lookback=setup_lookback)

    cd = compute_td_countdown_param(
        close, high, low,
        td['bull_setup_signal'],
        td['bear_setup_signal'],
        countdown_count=countdown_count,
        countdown_lookback=countdown_lookback)

    # ── V1: Setup only ─────────────────────────────────────
    long_entry_v1 = (
        (td['bull_setup_signal'] == 1) &
        (td['tdst_long_valid'] == 1)
    ).astype(int)
    long_carried_v1 = long_entry_v1.rolling(carry_setup, min_periods=1).max()
    v1_long  = (long_carried_v1 * td['tdst_long_valid']).clip(0, 1).shift(1).fillna(0)

    short_entry_v1 = (
        (td['bear_setup_signal'] == 1) &
        (td['tdst_short_valid'] == 1)
    ).astype(int)
    short_carried_v1 = short_entry_v1.rolling(carry_setup, min_periods=1).max()
    v1_short = (short_carried_v1 * td['tdst_short_valid']).clip(0, 1).shift(1).fillna(0)

    # ── V2: Countdown only ─────────────────────────────────
    long_entry_v2 = (
        (cd['bull_cd_signal'] == 1) &
        (td['tdst_long_valid'] == 1)
    ).astype(int)
    long_carried_v2 = long_entry_v2.rolling(carry_countdown, min_periods=1).max()
    v2_long  = (long_carried_v2 * td['tdst_long_valid']).clip(0, 1).shift(1).fillna(0)

    short_entry_v2 = (
        (cd['bear_cd_signal'] == 1) &
        (td['tdst_short_valid'] == 1)
    ).astype(int)
    short_carried_v2 = short_entry_v2.rolling(carry_countdown, min_periods=1).max()
    v2_short = (short_carried_v2 * td['tdst_short_valid']).clip(0, 1).shift(1).fillna(0)

    # ── V3: Combined 50/50 ────────────────────────────────
    v3_long = (
        (long_carried_v1 * 0.5 + long_carried_v2 * 0.5) *
        td['tdst_long_valid']
    ).clip(0, 1).shift(1).fillna(0)

    v3_short = (
        (short_carried_v1 * 0.5 + short_carried_v2 * 0.5) *
        td['tdst_short_valid']
    ).clip(0, 1).shift(1).fillna(0)

    return {
        'v1_long': v1_long,  'v1_short': v1_short,
        'v2_long': v2_long,  'v2_short': v2_short,
        'v3_long': v3_long,  'v3_short': v3_short,
        'bull_count':    td['bull_count'],
        'bear_count':    td['bear_count'],
        'bull_cd_count': cd['bull_cd_count'],
        'bear_cd_count': cd['bear_cd_count'],
    }


def backtest_config(
        close, high, low, returns,
        setup_count=9,
        countdown_count=13,
        setup_lookback=4,
        countdown_lookback=2,
        carry_setup=63,
        carry_countdown=126,
        cost_bps=5,
        variant='v1'):
    """
    Run a full equal-weight universe backtest for one parameter config.
    Returns a dict of performance metrics plus the raw return Series.

    variant: 'v1' (setup only), 'v2' (countdown only), 'v3' (combined)
    """
    sigs = build_signals_for_config(
        close, high, low,
        setup_count=setup_count,
        countdown_count=countdown_count,
        setup_lookback=setup_lookback,
        countdown_lookback=countdown_lookback,
        carry_setup=carry_setup,
        carry_countdown=carry_countdown)

    long_sig  = sigs[f'{variant}_long']
    short_sig = sigs[f'{variant}_short']

    # Long side
    lp     = long_sig.reindex(returns.index).fillna(0)
    ln     = apply_costs(returns, lp, cost_bps)
    n_long = long_sig.sum(axis=1).replace(0, np.nan)
    l_pnl  = (ln * lp).sum(axis=1)
    l_ret  = (l_pnl / n_long).fillna(0)

    # Short side
    sp      = short_sig.reindex(returns.index).fillna(0) * -1
    sn      = apply_costs(returns, sp, cost_bps)
    n_short = short_sig.sum(axis=1).replace(0, np.nan)
    s_pnl   = (sn * sp).sum(axis=1)
    s_ret   = (s_pnl / n_short).fillna(0)

    combined = ((l_ret + s_ret) / 2)

    stats = performance_summary(combined)
    stats['Signal Density Long']  = signal_density(long_sig)
    stats['Signal Density Short'] = signal_density(short_sig)
    stats['Setup Count']          = setup_count
    stats['Countdown Count']      = countdown_count
    stats['Setup Lookback']       = setup_lookback
    stats['Countdown Lookback']   = countdown_lookback
    stats['Variant']              = variant
    stats['ret_series']           = combined

    return stats


print("Signal builders & backtest function loaded")

# Cell 5 — Single Config Run

Run a single configuration to validate the engine before launching the grid search.

In [ ]:
# Cell 5: Single Config Run
# ============================================================
# ── CONTROLS — edit these to test a specific config ────────
SINGLE_SETUP_COUNT      = DEFAULT_SETUP_COUNT        # bars
SINGLE_COUNTDOWN_COUNT  = DEFAULT_COUNTDOWN_COUNT    # bars
SINGLE_SETUP_LOOKBACK   = DEFAULT_SETUP_LOOKBACK     # bars
SINGLE_CD_LOOKBACK      = DEFAULT_COUNTDOWN_LOOKBACK # bars
SINGLE_CARRY_SETUP      = DEFAULT_CARRY_SETUP        # bars
SINGLE_CARRY_COUNTDOWN  = DEFAULT_CARRY_COUNTDOWN    # bars
SINGLE_VARIANT          = 'v1'  # 'v1', 'v2', or 'v3'
# ──────────────────────────────────────────────────────────

import time
print(f"Running single config: Setup={SINGLE_SETUP_COUNT}, "
      f"Countdown={SINGLE_COUNTDOWN_COUNT}, "
      f"SetupLB={SINGLE_SETUP_LOOKBACK}, "
      f"CDLB={SINGLE_CD_LOOKBACK}, "
      f"Variant={SINGLE_VARIANT}")

t0 = time.time()
single_result = backtest_config(
    close, high, low, returns,
    setup_count        = SINGLE_SETUP_COUNT,
    countdown_count    = SINGLE_COUNTDOWN_COUNT,
    setup_lookback     = SINGLE_SETUP_LOOKBACK,
    countdown_lookback = SINGLE_CD_LOOKBACK,
    carry_setup        = SINGLE_CARRY_SETUP,
    carry_countdown    = SINGLE_CARRY_COUNTDOWN,
    cost_bps           = COST_BPS,
    variant            = SINGLE_VARIANT,
)
elapsed = time.time() - t0
print(f"\nDone in {elapsed:.1f}s")

print("\n" + "=" * 50)
print("SINGLE CONFIG RESULTS")
print("=" * 50)
for k, v in single_result.items():
    if k == 'ret_series':
        continue
    if isinstance(v, float):
        print(f"  {k:30s}: {v:.4f}")
    else:
        print(f"  {k:30s}: {v}")

# Equity curve for this config
fig, ax = plt.subplots(figsize=(14, 4))
cum = (1 + single_result['ret_series']).cumprod()
ax.plot(cum.index, cum, color='#1f77b4', linewidth=2)
ax.axhline(1, color='grey', linestyle='--', linewidth=0.8)
ax.set_title(
    f'Single Config Equity Curve — '
    f'Setup={SINGLE_SETUP_COUNT}, Countdown={SINGLE_COUNTDOWN_COUNT}, '
    f'SetupLB={SINGLE_SETUP_LOOKBACK}, CDLB={SINGLE_CD_LOOKBACK}, '
    f'Variant={SINGLE_VARIANT}\n'
    f'Sharpe: {single_result["Sharpe"]:.3f}  |  '
    f'Ann. Return: {single_result["Ann. Return"]:.2%}  |  '
    f'Max DD: {single_result["Max DD"]:.2%}',
    fontweight='bold', fontsize=10)
ax.set_ylabel('Cumulative Return')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('single_config.png', dpi=150)
plt.show()

# Cell 6 — Grid Search

Sweeps all combinations of `SETUP_COUNT_RANGE × COUNTDOWN_COUNT_RANGE`.

⚠️ **Runtime warning**: Each iteration requires a countdown loop per ticker. A 6×7 grid = 42 configs × ~5-15s each. Expect 5–10 minutes for the full grid.

In [ ]:
# Cell 6: Grid Search
# ============================================================
# ── CONTROLS ──────────────────────────────────────────────
GRID_VARIANTS        = ['v1', 'v2', 'v3']  # which variants to run
GRID_SETUP_LOOKBACK  = DEFAULT_SETUP_LOOKBACK
GRID_CD_LOOKBACK     = DEFAULT_COUNTDOWN_LOOKBACK
GRID_CARRY_SETUP     = DEFAULT_CARRY_SETUP
GRID_CARRY_COUNTDOWN = DEFAULT_CARRY_COUNTDOWN
# ──────────────────────────────────────────────────────────

import time

grid_combos = list(product(SETUP_COUNT_RANGE, COUNTDOWN_COUNT_RANGE))
total_runs  = len(grid_combos) * len(GRID_VARIANTS)

print(f"Grid search: {len(SETUP_COUNT_RANGE)} setup counts × "
      f"{len(COUNTDOWN_COUNT_RANGE)} countdown counts × "
      f"{len(GRID_VARIANTS)} variants = {total_runs} configs")
print(f"Setup range:    {SETUP_COUNT_RANGE}")
print(f"Countdown range:{COUNTDOWN_COUNT_RANGE}")
print(f"Variants:       {GRID_VARIANTS}")

grid_results = []
t_start = time.time()

for run_idx, (sc, cc) in enumerate(grid_combos):
    t_iter = time.time()
    for variant in GRID_VARIANTS:
        result = backtest_config(
            close, high, low, returns,
            setup_count        = sc,
            countdown_count    = cc,
            setup_lookback     = GRID_SETUP_LOOKBACK,
            countdown_lookback = GRID_CD_LOOKBACK,
            carry_setup        = GRID_CARRY_SETUP,
            carry_countdown    = GRID_CARRY_COUNTDOWN,
            cost_bps           = COST_BPS,
            variant            = variant,
        )
        grid_results.append(result)

    elapsed = time.time() - t_start
    done    = run_idx + 1
    remaining = (elapsed / done) * (len(grid_combos) - done)
    print(f"  [{done:3d}/{len(grid_combos)}] SC={sc:2d} CC={cc:2d}  "
          f"({time.time()-t_iter:.1f}s)  "
          f"ETA: {remaining/60:.1f}m")

total_time = time.time() - t_start
print(f"\nGrid search complete in {total_time/60:.1f} minutes")
print(f"{len(grid_results)} results")

# Build results table
grid_df = pd.DataFrame([
    {k: v for k, v in r.items() if k != 'ret_series'}
    for r in grid_results
])

grid_df_sorted = grid_df.sort_values('Sharpe', ascending=False)
print("\nTop 20 configurations by Sharpe:")
print(grid_df_sorted[
    ['Variant', 'Setup Count', 'Countdown Count',
     'Sharpe', 'Ann. Return', 'Max DD', 'Calmar',
     'Signal Density Long', 'Signal Density Short']
].head(20).round(3).to_string())

# Cell 7 — Results Heatmaps

In [ ]:
# Cell 7: Results Heatmaps
# ============================================================
# Show Sharpe, Ann. Return, Max DD, and Signal Density
# as 2D heatmaps across Setup Count x Countdown Count
# for each variant.

HEATMAP_METRIC = 'Sharpe'  # Change to 'Ann. Return', 'Max DD', 'Calmar', etc.

variants = grid_df['Variant'].unique()
n_variants = len(variants)
metrics_to_show = ['Sharpe', 'Ann. Return', 'Max DD', 'Signal Density Long']

fig, axes = plt.subplots(
    n_variants, len(metrics_to_show),
    figsize=(6 * len(metrics_to_show), 4.5 * n_variants))

for row, variant in enumerate(variants):
    vdf = grid_df[grid_df['Variant'] == variant]

    for col, metric in enumerate(metrics_to_show):
        ax = axes[row, col] if n_variants > 1 else axes[col]

        pivot = vdf.pivot(
            index='Setup Count',
            columns='Countdown Count',
            values=metric
        )

        vmin = pivot.values.min()
        vmax = pivot.values.max()

        # For Max DD lower is better — flip the colormap
        cmap = 'RdYlGn' if metric != 'Max DD' else 'RdYlGn_r'

        im = ax.imshow(
            pivot.values,
            cmap=cmap,
            aspect='auto',
            vmin=vmin, vmax=vmax,
            interpolation='nearest'
        )

        ax.set_xticks(range(len(pivot.columns)))
        ax.set_xticklabels(pivot.columns, fontsize=9)
        ax.set_yticks(range(len(pivot.index)))
        ax.set_yticklabels(pivot.index, fontsize=9)

        # Annotate cells
        for i in range(len(pivot.index)):
            for j in range(len(pivot.columns)):
                val = pivot.iloc[i, j]
                if not np.isnan(val):
                    # Highlight the original 9/13 cell
                    sc_val = pivot.index[i]
                    cc_val = pivot.columns[j]
                    is_orig = (sc_val == 9 and cc_val == 13)
                    fmt = f'{val:.3f}' if metric in ['Sharpe', 'Calmar'] else f'{val:.2%}'
                    ax.text(
                        j, i, fmt,
                        ha='center', va='center',
                        fontsize=8,
                        fontweight='bold' if is_orig else 'normal',
                        color='white' if abs((val - vmin) / (vmax - vmin + 1e-8)) > 0.7 else 'black'
                    )

        plt.colorbar(im, ax=ax, shrink=0.7)
        ax.set_xlabel('Countdown Count', fontsize=9)
        ax.set_ylabel('Setup Count', fontsize=9)
        ax.set_title(f'{variant.upper()} — {metric}\n(bold = original 9/13)',
                     fontweight='bold', fontsize=10)

fig.suptitle(
    'TD Count-Length Grid Search\n'
    'Rows = Variant   Columns = Metric\n'
    'X-axis = Countdown bars   Y-axis = Setup bars',
    fontsize=13, fontweight='bold', y=1.01)

plt.tight_layout()
plt.savefig('td_grid_heatmaps.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: td_grid_heatmaps.png")

# Cell 8 — Sharpe Profiles (line plots across count ranges)

In [ ]:
# Cell 8: Sharpe Profiles
# ============================================================
# For each variant, show how Sharpe varies as you slide
# setup count (fixing countdown) and vice versa.
# This reveals whether 9 and 13 sit at a local maximum
# or on a slope.

fig, axes = plt.subplots(2, len(variants), figsize=(7 * len(variants), 10))

for col, variant in enumerate(variants):
    vdf = grid_df[grid_df['Variant'] == variant]

    # ── Top row: Sharpe vs Setup Count (one line per Countdown Count)
    ax = axes[0, col] if len(variants) > 1 else axes[0]
    for cc in sorted(vdf['Countdown Count'].unique()):
        sub = vdf[vdf['Countdown Count'] == cc].sort_values('Setup Count')
        is_orig_cc = (cc == 13)
        ax.plot(
            sub['Setup Count'],
            sub['Sharpe'],
            marker='o',
            linewidth=2.0 if is_orig_cc else 1.0,
            label=f'CD={cc}' + (' ◄ orig' if is_orig_cc else '')
        )
    ax.axvline(9, color='black', linestyle=':', linewidth=1.2, alpha=0.5, label='Setup=9 (orig)')
    ax.set_xlabel('Setup Count (bars)', fontsize=10)
    ax.set_ylabel('Sharpe Ratio', fontsize=10)
    ax.set_title(f'{variant.upper()} — Sharpe vs Setup Count\n(one line per Countdown Count)',
                 fontweight='bold', fontsize=10)
    ax.legend(fontsize=7, ncol=2)
    ax.grid(True, alpha=0.3)

    # ── Bottom row: Sharpe vs Countdown Count (one line per Setup Count)
    ax = axes[1, col] if len(variants) > 1 else axes[1]
    for sc in sorted(vdf['Setup Count'].unique()):
        sub = vdf[vdf['Setup Count'] == sc].sort_values('Countdown Count')
        is_orig_sc = (sc == 9)
        ax.plot(
            sub['Countdown Count'],
            sub['Sharpe'],
            marker='o',
            linewidth=2.0 if is_orig_sc else 1.0,
            label=f'Setup={sc}' + (' ◄ orig' if is_orig_sc else '')
        )
    ax.axvline(13, color='black', linestyle=':', linewidth=1.2, alpha=0.5, label='CD=13 (orig)')
    ax.set_xlabel('Countdown Count (bars)', fontsize=10)
    ax.set_ylabel('Sharpe Ratio', fontsize=10)
    ax.set_title(f'{variant.upper()} — Sharpe vs Countdown Count\n(one line per Setup Count)',
                 fontweight='bold', fontsize=10)
    ax.legend(fontsize=7, ncol=2)
    ax.grid(True, alpha=0.3)

fig.suptitle(
    'Sharpe Profiles — How sensitive are results to count lengths?\n'
    'Flat = robust   Peaked = overfit   Slope = better values exist elsewhere',
    fontsize=13, fontweight='bold', y=1.01)

plt.tight_layout()
plt.savefig('td_sharpe_profiles.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: td_sharpe_profiles.png")

# Cell 9 — Best Config Deep Dive

In [ ]:
# Cell 9: Best Config Deep Dive
# ============================================================
# Compares the best-found config against the original 9/13.
# Shows equity curves, drawdowns, and per-ticker Sharpe.

# ── CONTROLS ──────────────────────────────────────────────
DEEP_DIVE_VARIANT = 'v1'  # which variant to compare
# ──────────────────────────────────────────────────────────

# Find best config for the chosen variant
vdf_best = grid_df[
    (grid_df['Variant'] == DEEP_DIVE_VARIANT) &
    grid_df['Sharpe'].notna()
].sort_values('Sharpe', ascending=False)

best_row = vdf_best.iloc[0]
best_sc  = int(best_row['Setup Count'])
best_cc  = int(best_row['Countdown Count'])

print(f"Best config for {DEEP_DIVE_VARIANT.upper()}:")
print(f"  Setup Count    : {best_sc}")
print(f"  Countdown Count: {best_cc}")
print(f"  Sharpe         : {best_row['Sharpe']:.3f}")
print(f"  Ann. Return    : {best_row['Ann. Return']:.2%}")
print(f"  Max DD         : {best_row['Max DD']:.2%}")

# Retrieve the stored return series
best_ret_idx = next(
    i for i, r in enumerate(grid_results)
    if (r['Variant'] == DEEP_DIVE_VARIANT and
        r['Setup Count'] == best_sc and
        r['Countdown Count'] == best_cc)
)
best_ret = grid_results[best_ret_idx]['ret_series']

# Original 9/13 return series
orig_idx = next(
    i for i, r in enumerate(grid_results)
    if (r['Variant'] == DEEP_DIVE_VARIANT and
        r['Setup Count'] == 9 and
        r['Countdown Count'] == 13)
)
orig_ret = grid_results[orig_idx]['ret_series']
orig_stats = grid_results[orig_idx]

print(f"\nOriginal 9/13 config for {DEEP_DIVE_VARIANT.upper()}:")
print(f"  Sharpe     : {orig_stats['Sharpe']:.3f}")
print(f"  Ann. Return: {orig_stats['Ann. Return']:.2%}")
print(f"  Max DD     : {orig_stats['Max DD']:.2%}")

# ── Dashboard ─────────────────────────────────────────────
fig = plt.figure(figsize=(18, 14))
gs  = gridspec.GridSpec(2, 2, figure=fig, hspace=0.4, wspace=0.32)

# Panel 1: Equity curves
ax1 = fig.add_subplot(gs[0, :])
cum_best = (1 + best_ret).cumprod()
cum_orig = (1 + orig_ret).cumprod()
ax1.plot(cum_best.index, cum_best,
         color='#2ca02c', linewidth=2.5,
         label=f'Best: Setup={best_sc}, CD={best_cc}  SR={best_row["Sharpe"]:.3f}')
ax1.plot(cum_orig.index, cum_orig,
         color='#1f77b4', linewidth=2.0,
         linestyle='--',
         label=f'Original: Setup=9, CD=13  SR={orig_stats["Sharpe"]:.3f}')
ax1.axhline(1, color='grey', linestyle=':', linewidth=0.8)
ax1.set_title(
    f'Best vs Original Config — {DEEP_DIVE_VARIANT.upper()}\n'
    f'Equal-weight universe, combined long + short',
    fontweight='bold', fontsize=11)
ax1.set_ylabel('Cumulative Return')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Panel 2: Drawdown comparison
ax2 = fig.add_subplot(gs[1, 0])
dd_best = cum_best / cum_best.cummax() - 1
dd_orig = cum_orig / cum_orig.cummax() - 1
ax2.fill_between(dd_best.index, dd_best, 0, alpha=0.4, color='#2ca02c', label='Best')
ax2.fill_between(dd_orig.index, dd_orig, 0, alpha=0.3, color='#1f77b4', label='Original')
ax2.set_title('Drawdown Comparison', fontweight='bold', fontsize=10)
ax2.set_ylabel('Drawdown')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Panel 3: Annual returns comparison
ax3 = fig.add_subplot(gs[1, 1])
ann_best = best_ret.resample('YE').apply(lambda x: (1+x).prod()-1)
ann_orig = orig_ret.resample('YE').apply(lambda x: (1+x).prod()-1)
years    = ann_best.index.year
x        = np.arange(len(years))
w        = 0.35
ax3.bar(x - w/2, ann_best.values, w, label='Best', color='#2ca02c', alpha=0.8, edgecolor='black', linewidth=0.4)
ax3.bar(x + w/2, ann_orig.values, w, label='Original', color='#1f77b4', alpha=0.7, edgecolor='black', linewidth=0.4)
ax3.axhline(0, color='black', linewidth=0.8)
ax3.set_xticks(x)
ax3.set_xticklabels(years, rotation=30, ha='right', fontsize=8)
ax3.set_title('Annual Returns', fontweight='bold', fontsize=10)
ax3.set_ylabel('Annual Return')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3, axis='y')

fig.suptitle(
    f'Deep Dive — {DEEP_DIVE_VARIANT.upper()}  |  '
    f'Best: Setup={best_sc} CD={best_cc}  vs  Original: Setup=9 CD=13',
    fontsize=13, fontweight='bold', y=1.01)

plt.savefig('td_best_vs_original.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: td_best_vs_original.png")

# Cell 10 — Per-Ticker Breakdown

In [ ]:
# Cell 10: Per-Ticker Breakdown
# ============================================================
# Compare per-ticker Sharpe between best and original config.
# Reveals which instruments drive the improvement.

# ── CONTROLS ──────────────────────────────────────────────
TICKER_BREAKDOWN_VARIANT = DEEP_DIVE_VARIANT
# ──────────────────────────────────────────────────────────

def per_ticker_sharpe_for_config(
        close, high, low, returns,
        setup_count, countdown_count,
        setup_lookback=4,
        countdown_lookback=2,
        carry_setup=63,
        carry_countdown=126,
        cost_bps=5,
        variant='v1'):
    """Compute Sharpe per ticker for a given config."""
    sigs = build_signals_for_config(
        close, high, low,
        setup_count=setup_count,
        countdown_count=countdown_count,
        setup_lookback=setup_lookback,
        countdown_lookback=countdown_lookback,
        carry_setup=carry_setup,
        carry_countdown=carry_countdown)

    long_sig  = sigs[f'{variant}_long']
    short_sig = sigs[f'{variant}_short']

    sharpes = {}
    for ticker in TICKERS:
        if ticker not in close.columns:
            continue
        lp  = long_sig[ticker].reindex(returns.index).fillna(0)
        sp  = short_sig[ticker].reindex(returns.index).fillna(0) * -1
        ln  = apply_costs(returns[[ticker]], lp.to_frame(), cost_bps)[ticker]
        sn  = apply_costs(returns[[ticker]], sp.to_frame(), cost_bps)[ticker]
        lr  = (ln * lp).fillna(0)
        sr  = (sn * sp).fillna(0)
        cr  = ((lr + sr) / 2).dropna()
        mu  = cr.mean()
        sd  = cr.std()
        sharpes[ticker] = (mu / sd * np.sqrt(252) if sd > 0 else np.nan)
    return pd.Series(sharpes)


print("Computing per-ticker Sharpe for best and original configs...")
best_ticker_sr = per_ticker_sharpe_for_config(
    close, high, low, returns,
    setup_count=best_sc,
    countdown_count=best_cc,
    variant=TICKER_BREAKDOWN_VARIANT)

orig_ticker_sr = per_ticker_sharpe_for_config(
    close, high, low, returns,
    setup_count=9,
    countdown_count=13,
    variant=TICKER_BREAKDOWN_VARIANT)

comparison = pd.DataFrame({
    f'Best (SC={best_sc},CC={best_cc})': best_ticker_sr,
    'Original (SC=9,CC=13)':             orig_ticker_sr,
    'Delta':                              best_ticker_sr - orig_ticker_sr,
})

print("\nPer-Ticker Sharpe — Best vs Original:")
print(comparison.round(3).to_string())

# Chart
fig, ax = plt.subplots(figsize=(14, 5))
tickers = comparison.index
x = np.arange(len(tickers))
w = 0.3
ax.bar(x - w, comparison.iloc[:, 0], w,
       label=comparison.columns[0],
       color='#2ca02c', alpha=0.85, edgecolor='black', linewidth=0.4)
ax.bar(x,     comparison.iloc[:, 1], w,
       label=comparison.columns[1],
       color='#1f77b4', alpha=0.75, edgecolor='black', linewidth=0.4)
ax.bar(x + w, comparison['Delta'], w,
       label='Delta (Best - Orig)',
       color='#ff7f0e', alpha=0.85, edgecolor='black', linewidth=0.4)
ax.axhline(0, color='black', linewidth=0.8)
ax.set_xticks(x)
ax.set_xticklabels(tickers, fontsize=10)
ax.set_title(
    f'Per-Ticker Sharpe — {TICKER_BREAKDOWN_VARIANT.upper()} | '
    f'Best SC={best_sc} CC={best_cc} vs Original SC=9 CC=13',
    fontweight='bold', fontsize=10)
ax.set_ylabel('Sharpe Ratio')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('td_per_ticker_comparison.png', dpi=150)
plt.show()
print("Saved: td_per_ticker_comparison.png")

# Cell 11 — Summary & Export

Print a summary table of all results and export to CSV for offline analysis.

In [ ]:
# Cell 11: Summary & Export
# ============================================================

print("=" * 70)
print("TD COUNT-LENGTH SANDBOX — FULL RESULTS")
print("=" * 70)

for variant in sorted(grid_df['Variant'].unique()):
    print(f"\n{'─' * 70}")
    print(f"Variant: {variant.upper()}")
    print(f"{'─' * 70}")

    vdf = grid_df[grid_df['Variant'] == variant].sort_values('Sharpe', ascending=False)
    orig_row = vdf[
        (vdf['Setup Count'] == 9) &
        (vdf['Countdown Count'] == 13)
    ]

    print(f"  Original 9/13 rank: "
          f"{vdf.index.get_loc(orig_row.index[0]) + 1 if len(orig_row) > 0 else 'N/A'} "
          f"of {len(vdf)}")

    print(f"\n  Top 5:")
    print(vdf[['Setup Count', 'Countdown Count', 'Sharpe', 'Ann. Return', 'Max DD', 'Calmar']]
          .head(5).round(3).to_string())

# Export
export_df = grid_df.drop(columns=['ret_series'], errors='ignore')
export_df.to_csv('td_grid_results.csv', index=False)
print("\n\nFull results exported to: td_grid_results.csv")

print("\n" + "=" * 70)
print("KEY QUESTIONS TO ASK OF THESE RESULTS:")
print("=" * 70)
print("""
  1. Is 9/13 at a local Sharpe maximum, or is there a better plateau?
     → Check the heatmaps (Cell 7). A smooth peak = likely overfit.
        A broad plateau = genuine structural robustness.

  2. Are the Sharpe profiles flat or sloped?
     → Check the line plots (Cell 8). Flat profiles = the count
        doesn't matter much. Steep slopes = you're in a sensitive zone.

  3. Does the best config beat 9/13 consistently across tickers?
     → Check Cell 10. If improvement is concentrated in 1-2 tickers
        it's likely noise.

  4. Does signal density change materially across configs?
     → Lower setup count = more signals = lower average quality.
        Higher countdown count = fewer signals = longer wait for entry.

  5. Is the improvement in-sample or structural?
     → Re-run with a different START_DATE or split the sample in half.
        If the best config moves materially, you're overfitting.
""")